# Webscarping Goodreads

This file is used to webscrape the required book data from the goodread.com List of "Books that everyone should have read at least once", using beautiful soup.
The list contains 24,529 books.

## Import Libraries

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import math
import re

## Request URL

In [2]:
books = requests.get("https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once?page={pagenr}")
print("books:", books.status_code)

books: 200


## Fetch data

In [ ]:
#try first with only titles on all pages

In [ ]:
title=[]

for link in range(1:100):
    try:
        r = requests.get(f'https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once?page={link}')
        soup = BeautifulSoup(r.content, 'html.parser')
        titles += soup.find#('div', attrs={'class': 'lister-item-content'})
    except:
        return('none')
        continue

In [ ]:
titles =[]
series=[]
authors=[]
genres=[]
descriptions=[]
years=[]
pages=[]
languages=[]
av_ratings=[]
number_ratings=[]
cover_image_link=[]






# book title - Done


#book series
def get_series_name(soup):
    import re
    series = soup.find(id="bookSeries").find("a")
    if series:
        series_name = re.search(r'\((.*?)\)', series.text).group(1)
        return series_name
    else:
        return ""

#book genres 

#book author - Done

#book desription -Done

#book year published - Done

#number of pages - Done 

#language - Done 

#isbn

#av rating

#1 star ratings, 2 star ratings ..

#number of ratings

#book cover link

In [112]:
books = requests.get("https://www.goodreads.com/book/show/3450744-nudge")
print("books:", books.status_code)
soup = BeautifulSoup(books.content, 'html.parser')

books: 200


In [14]:
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>

<html class="desktop">
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# good_reads: http://ogp.me/ns/fb/good_reads#">
<title>The First Days (As the World Dies, #1) by Rhiannon Frater</title>
<meta content="The First Days book. Read 1,141 reviews from the world's largest community for readers. Katie is driving to work one beautiful day when a dead man jumps ..." name="description"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://www.goodreads.com/book/show/9648068-the-first-days" rel="canonical"/>
<meta content="2415071772" property="fb:app_id"/>
<meta content="books.book" property="og:type"/>
<meta content="The First Days (As the World Dies, #1)" property="og:title"/>
<meta content="Katie is driving to work one beautiful day when a dead man jumps into her car and tries to eat her.  That same morning, Jenni opens a bed..." property="og:description"/>
<meta content="https://i.gr-assets.com/images/S/com

### Book Title

In [113]:
def get_book_title(soup):
    if soup.find(id='bookTitle'):
        btitle = soup.find(id="bookTitle").get_text(strip=True)
        return btitle
    return ''

In [114]:
get_book_title(soup)

'Nudge: Improving Decisions About Health, Wealth, and Happiness'

### Book Description

In [115]:
def get_book_description(soup):
    bdescription= ''
    if soup.find(id="description").find(style="display:none"):
        bdescription +=soup.find(id="description").find(style="display:none").get_text(strip=True)
        return bdescription
    return ''

In [116]:
get_book_description(soup)

'From the winner of the 2017 Nobel Prize in Economics, Richard H. Thaler, and Cass R. Sunstein: a revelatory look at how we make decisionsNew York Times bestsellerNamed a Best Book of the Year byThe Economistand theFinancial TimesEvery day we make choices—about what to buy or eat, about financial investments or our children’s health and education, even about the causes we champion or the planet itself. Unfortunately, we often choose poorly. Nudge is about how we make these choices and how we can make better ones. Using dozens of eye-opening examples and drawing on decades of behavioral science research, Nobel Prize winner Richard H. Thaler and Harvard Law School professor Cass R. Sunstein show that no choice is ever presented to us in a neutral way, and that we are all susceptible to biases that can lead us to make bad decisions. But by knowing how people think, we can use sensible “choice architecture” to nudge people toward the best decisions for ourselves, our families, and our soci

### Book Author

In [184]:
def get_book_authors1(soup):
    if soup.find(id="bookAuthors").find('a').find(itemprop='name'):  #find('a', attrs={'class': 'authorName'}).find(itemprop='name'):
        authorss=soup.find(id="bookAuthors").find('a').find(itemprop='name').get_text(strip=True)
        return authorss
    return ''
#gives only first author

In [185]:
get_book_authors1(soup)

'Richard H. Thaler'

In [186]:
#DONE - This function gives string with all the authors, sperated by comma
def get_book_authors(soup):
    if soup.find(id="bookAuthors").find_all(itemprop='name'):  #find('a', attrs={'class': 'authorName'}).find(itemprop='name'):
        authors=soup.find(id="bookAuthors").find_all(itemprop='name')#.find(itemprop='name')
        
        authors_all=''
        for i in authors:
            authors_all+=(i.get_text())+','
        return(authors_all)
    return ''

In [187]:
get_book_authors(soup)

'Richard H. Thaler,Cass R. Sunstein,'

### Number of pages

In [231]:
def get_book_pages(soup):
    if soup.find(id="details").find(itemprop='numberOfPages'):  #find('a', attrs={'class': 'authorName'}).find(itemprop='name'):
        pages=soup.find(id="details").find(itemprop='numberOfPages').get_text(strip=True)#.find(itemprop='name')
        return int(pages.split()[0])
    return ''

In [232]:
get_book_pages(soup)

260

### Date first published

In [233]:
def get_book_year(soup):
    if soup.find('nobr', attrs={'class':'greyText'}):
        year_published=soup.find('nobr', attrs={'class':'greyText'}).get_text(strip=True)
        #print(year_published)
        return re.search('(\d{4})', year_published).group(1) #search for year: number with 4 digits
    return ''

In [234]:
get_book_year(soup)

'2008'

### Language

In [235]:
def get_book_language(soup):
    if soup.find(itemprop="inLanguage"):
        language=soup.find(itemprop="inLanguage").get_text(strip=True)
        return language
    return ''

In [236]:
get_book_language(soup)

'English'

In [119]:
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>

<html class="desktop">
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# good_reads: http://ogp.me/ns/fb/good_reads#">
<title>Nudge: Improving Decisions About Health, Wealth, and Happiness by Richard H. Thaler</title>
<meta content="Nudge book. Read 3,534 reviews from the world's largest community for readers. From the winner of the 2017 Nobel Prize in Economics, Richard H. Thaler, a..." name="description"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://www.goodreads.com/book/show/3450744-nudge" rel="canonical"/>
<meta content="2415071772" property="fb:app_id"/>
<meta content="books.book" property="og:type"/>
<meta content="Nudge" property="og:title"/>
<meta content=" From the winner of the 2017 Nobel Prize in Economics, Richard H. Thaler, and Cass R. Sunstein: a revelatory look at how we make decision..." property="og:description"/>
<meta content="https://i.gr-assets.com/images/S/compressed.photo.go

In [66]:
# This function finds the book series 

def get_series_name(soup):
    import re
    series = soup.find(id="bookSeries").find("a")
    if series:
        series_name = re.search(r'\((.*?)\)', series.text).group(1)
        return series_name
    else:
        return ""

In [67]:
get_series_name(soup)

'As the World Dies #1'

In [12]:
# This function returns the number of pages def get_num_pages(soup):
def get_num_pages(soup):
    if soup.find('span', {'itemprop': 'numberOfPages'}):
        num_pages = soup.find('span', {'itemprop': 'numberOfPages'}).text.strip()
        return int(num_pages.split()[0])
    return ''

In [13]:
get_num_pages(soup)

331

In [ ]:
def pages(soup):
    if soup.find(('div', attrs={'class': 'FeaturedDetails'}).find('p').get_text()split()[-2])
    